# Exchange rates from CNB 

(Czech National Bank)

## Sources:

- [CNB FAQ: Ex rates, data formats, URLs](https://www.cnb.cz/cs/casto-kladene-dotazy/Kurzy-devizoveho-trhu-na-www-strankach-CNB/)


In [584]:
import pandas as pd

In [585]:
URL_annual_exrates_root = 'https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/rok.txt?rok='

In [586]:
required_year = 2022

Let's combine the year into URL:

In [587]:
full_URL = URL_annual_exrates_root + str(required_year)

And read data from CNB. According to their [documentation](https://www.cnb.cz/cs/casto-kladene-dotazy/Kurzy-devizoveho-trhu-na-www-strankach-CNB/), the file consist of 
- first row as a header (containing multiplier and currency, but we will deal with that in the next step)
- the column 0 is the date in DD.MM.YYYY format
- other columns are numbers, using comma (',') as decimal separator

In [588]:
df_exrates = pd.read_csv(full_URL, sep='|', header=0, parse_dates=[0], date_format = '%d.%m.%Y', decimal=',')
df_exrates
# df_exrates
# pd.to_numeric(df_exrates['1 AUD'].str.replace(',','.'), errors='coerce')
for col in df_exrates.columns[1:]:
    # print(df_exrates[col])
    # print(pd.to_numeric(df_exrates[col].str.replace(',','.'), errors='coerce'))
    # df_exrates[col] = pd.to_numeric(df_exrates[col].str.replace(',','.'), errors='coerce')

    # pd.to_numeric(df_exrates[col].str.replace(',','.'), errors='coerce')
    pass
df_exrates

,Datum,1 AUD,1 BGN,1 BRL,1 CAD,1 CHF,1 CNY,1 DKK,1 EUR,1 GBP,...,1 PLN,1 RON,100 RUB,1 SEK,1 SGD,100 THB,1 TRY,1 USD,1 XDR,1 ZAR
0,03.01.2022,"15,818","12,690","3,906","17,210","23,931","3,441","3,337","24,820","29,502",...,"5,408","5,016","29,362","2,410","16,189","65,897","1,646","21,860","30,595",1.382
1,04.01.2022,"15,784","12,655","3,857","17,211","23,903","3,441","3,328","24,750","29,599",...,"5,419","5,002","29,146","2,407","16,166","65,862","1,635","21,944","30,744",1.367
2,05.01.2022,"15,764","12,567","3,831","17,076","23,722","3,414","3,305","24,580","29,426",...,"5,382","4,970","28,697","2,397","16,040","65,439","1,613","21,739","30,359",1.372
3,06.01.2022,"15,547","12,542","3,810","16,977","23,600","3,398","3,297","24,530","29,345",...,"5,378","4,962","28,356","2,375","15,944","64,656","1,578","21,679","30,332",1.379
4,07.01.2022,"15,462","12,495","3,798","17,000","23,450","3,391","3,286","24,440","29,291",...,"5,372","4,942","28,659","2,376","15,912","64,204","1,554","21,626","30,243",1.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,23.12.2022,"15,292","12,397","4,423","16,799","24,572","3,268","3,260","24,245","27,546",...,"5,223","4,942","2,183","16,912","65,810","1,222","22,825","30,390","1,340",NaN
249,27.12.2022,"15,383","12,390","4,330","16,864","24,541","3,278","3,262","24,260","27,467",...,"5,197","4,925","2,180","16,962","65,959","1,220","22,831","30,384","1,324",NaN
250,28.12.2022,"15,485","12,371","4,323","16,889","24,589","3,268","3,261","24,250","27,539",...,"5,157","4,903","2,184","16,934","65,641","1,218","22,796","30,337","1,326",NaN
251,29.12.2022,"15,255","12,368","4,373","16,711","24,586","3,263","3,253","24,190","27,325",...,"5,163","4,887","2,168","16,849","65,604","1,214","22,718","30,234","1,330",NaN


**IMPORTANT:** In some years, like 2009 and 2022, the list of currencies has changed in the middle of the year. If it happens, it is necessary to split the year amd read each part independently.

In [589]:
if df_exrates['Datum'].dtype == 'object':
    print(f'Incorrect types')
    ser_new_headers = pd.Series(df_exrates[df_exrates['Datum']== 'Datum'].index)
    skip_rows = 0
    partial_dfs = list()
    for hdr_line in ser_new_headers:
        df_exrates_partial = pd.read_csv(full_URL, sep='|', header=0, parse_dates=[0], date_format = '%d.%m.%Y', decimal=',', nrows=hdr_line-skip_rows, skiprows=skip_rows)
        partial_dfs.append(df_exrates_partial)
        print(f'{hdr_line = }\n{df_exrates_partial.dtypes[0:4]}')
        skip_rows = hdr_line+1

    df_exrates_partial = pd.read_csv(full_URL, sep='|', header=0, parse_dates=[0], date_format = '%d.%m.%Y', decimal=',', skiprows=skip_rows)
    partial_dfs.append(df_exrates_partial)

    df_exrates =  pd.concat(partial_dfs, ignore_index=True)

df_exrates.to_csv('temp.csv')


Incorrect types
hdr_line = 42
Datum    datetime64[ns]
1 AUD           float64
1 BGN           float64
1 BRL           float64
dtype: object


In [590]:
df_exrates_partial = pd.read_csv(full_URL, sep='|', header=0, parse_dates=[0], date_format = '%d.%m.%Y', decimal=',', skiprows=skip_rows)

Let's deal with column names. We should split the currency name from multiplier and put it into new table.

In [591]:
df_currencies = pd.DataFrame()
df_currencies['orig_name'] = df_exrates.columns[1:]
df_currencies[['multiplier', 'currency']] = df_currencies['orig_name'].str.split(pat = ' ', expand=True)
df_currencies

,orig_name,multiplier,currency
0,1 AUD,1,AUD
1,1 BGN,1,BGN
2,1 BRL,1,BRL
3,1 CAD,1,CAD
4,1 CHF,1,CHF
5,1 CNY,1,CNY
6,1 DKK,1,DKK
7,1 EUR,1,EUR
8,1 GBP,1,GBP
9,1 HKD,1,HKD


In [592]:
df_currencies.set_index('orig_name', inplace=True)
df_currencies

,multiplier,currency
orig_name,,
1 AUD,1,AUD
1 BGN,1,BGN
1 BRL,1,BRL
1 CAD,1,CAD
1 CHF,1,CHF
1 CNY,1,CNY
1 DKK,1,DKK
1 EUR,1,EUR
1 GBP,1,GBP


In [593]:

new_column_names =  pd.Series(df_currencies['currency'],index=df_currencies.index).to_dict()
new_column_names


{'1 AUD': 'AUD',
 '1 BGN': 'BGN',
 '1 BRL': 'BRL',
 '1 CAD': 'CAD',
 '1 CHF': 'CHF',
 '1 CNY': 'CNY',
 '1 DKK': 'DKK',
 '1 EUR': 'EUR',
 '1 GBP': 'GBP',
 '1 HKD': 'HKD',
 '1 HRK': 'HRK',
 '100 HUF': 'HUF',
 '1000 IDR': 'IDR',
 '1 ILS': 'ILS',
 '100 INR': 'INR',
 '100 ISK': 'ISK',
 '100 JPY': 'JPY',
 '100 KRW': 'KRW',
 '1 MXN': 'MXN',
 '1 MYR': 'MYR',
 '1 NOK': 'NOK',
 '1 NZD': 'NZD',
 '100 PHP': 'PHP',
 '1 PLN': 'PLN',
 '1 RON': 'RON',
 '100 RUB': 'RUB',
 '1 SEK': 'SEK',
 '1 SGD': 'SGD',
 '100 THB': 'THB',
 '1 TRY': 'TRY',
 '1 USD': 'USD',
 '1 XDR': 'XDR',
 '1 ZAR': 'ZAR'}

In [594]:
df_exrates.rename(columns=new_column_names, inplace=True)
df_exrates.set_index('Datum', inplace=True)
df_exrates


,AUD,BGN,BRL,CAD,CHF,CNY,DKK,EUR,GBP,HKD,...,PLN,RON,RUB,SEK,SGD,THB,TRY,USD,XDR,ZAR
Datum,,,,,,,,,,,,,,,,,,,,,
2022-01-03,15.818,12.690,3.906,17.210,23.931,3.441,3.337,24.820,29.502,2.803,...,5.408,5.016,29.362,2.410,16.189,65.897,1.646,21.860,30.595,1.382
2022-01-04,15.784,12.655,3.857,17.211,23.903,3.441,3.328,24.750,29.599,2.815,...,5.419,5.002,29.146,2.407,16.166,65.862,1.635,21.944,30.744,1.367
2022-01-05,15.764,12.567,3.831,17.076,23.722,3.414,3.305,24.580,29.426,2.789,...,5.382,4.970,28.697,2.397,16.040,65.439,1.613,21.739,30.359,1.372
2022-01-06,15.547,12.542,3.810,16.977,23.600,3.398,3.297,24.530,29.345,2.779,...,5.378,4.962,28.356,2.375,15.944,64.656,1.578,21.679,30.332,1.379
2022-01-07,15.462,12.495,3.798,17.000,23.450,3.391,3.286,24.440,29.291,2.772,...,5.372,4.942,28.659,2.376,15.912,64.204,1.554,21.626,30.243,1.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,15.292,12.397,4.423,16.799,24.572,3.268,3.260,24.245,27.546,2.925,...,5.223,4.942,NaN,2.183,16.912,65.810,1.222,22.825,30.390,1.340
2022-12-27,15.383,12.390,4.330,16.864,24.541,3.278,3.262,24.260,27.467,2.927,...,5.197,4.925,NaN,2.180,16.962,65.959,1.220,22.831,30.384,1.324
2022-12-28,15.485,12.371,4.323,16.889,24.589,3.268,3.261,24.250,27.539,2.925,...,5.157,4.903,NaN,2.184,16.934,65.641,1.218,22.796,30.337,1.326


I can either use this table and save it to csv or convert columns into rows to get following structure:

|datum|currency|exrate|
|-|-|-|
|2013-01-02|AUD|19.965|
|2013-01-02|BGN|12.898|



In [595]:
df_currencies.to_csv('currencies.csv')
# saving as a flat file, with a column per currency
df_exrates.to_csv('ex_rates_wide.csv',index=True, index_label='datum')

In [596]:
# df_exrates.reset_index(inplace=True)
df_exrates_depivoted = df_exrates.reset_index().melt(id_vars=['Datum'],var_name = 'currency', value_name='ex_rate'  )
df_exrates_depivoted

,Datum,currency,ex_rate
0,2022-01-03,AUD,15.818
1,2022-01-04,AUD,15.784
2,2022-01-05,AUD,15.764
3,2022-01-06,AUD,15.547
4,2022-01-07,AUD,15.462
...,...,...,...
8311,2022-12-23,ZAR,1.340
8312,2022-12-27,ZAR,1.324
8313,2022-12-28,ZAR,1.326
8314,2022-12-29,ZAR,1.330


In [597]:
df_exrates_depivoted.to_csv('ex_rates_long.csv', index=False)

### Q: What if I want to add Saturdays and Sundays into the dataset?

It would be better to add that to the original dataset as new rows, based on the value from Friday. Let's try that.

In [598]:
min_date = df_exrates.index.min()
min_date
max_date = df_exrates.index.max()
[max_date, max_date.day_of_week]

[Timestamp('2022-12-30 00:00:00'), 4]

In [599]:
df_calendar = pd.DataFrame({"Date": pd.date_range(min_date, max_date)})
df_calendar

,Date
0,2022-01-03
1,2022-01-04
2,2022-01-05
3,2022-01-06
4,2022-01-07
...,...
357,2022-12-26
358,2022-12-27
359,2022-12-28
360,2022-12-29


In [600]:
df_exrates[df_exrates.index <= '2023-01-08'].index.max()

Timestamp('2022-12-30 00:00:00')

In [601]:
df_calendar.set_index('Date', inplace=True)
df_calendar

""
Date
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
...
2022-12-26
2022-12-27
2022-12-28


In [602]:
df_calendar

""
Date
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
...
2022-12-26
2022-12-27
2022-12-28


In [603]:
# df_exrates.reset_index(inplace=True)
# df_exrates.drop(['index', 'level_0'], axis=1, inplace=True)
if 'Datum' not in df_exrates.columns:
    df_exrates.reset_index(inplace=True)

df_exrates

,Datum,AUD,BGN,BRL,CAD,CHF,CNY,DKK,EUR,GBP,...,PLN,RON,RUB,SEK,SGD,THB,TRY,USD,XDR,ZAR
0,2022-01-03,15.818,12.690,3.906,17.210,23.931,3.441,3.337,24.820,29.502,...,5.408,5.016,29.362,2.410,16.189,65.897,1.646,21.860,30.595,1.382
1,2022-01-04,15.784,12.655,3.857,17.211,23.903,3.441,3.328,24.750,29.599,...,5.419,5.002,29.146,2.407,16.166,65.862,1.635,21.944,30.744,1.367
2,2022-01-05,15.764,12.567,3.831,17.076,23.722,3.414,3.305,24.580,29.426,...,5.382,4.970,28.697,2.397,16.040,65.439,1.613,21.739,30.359,1.372
3,2022-01-06,15.547,12.542,3.810,16.977,23.600,3.398,3.297,24.530,29.345,...,5.378,4.962,28.356,2.375,15.944,64.656,1.578,21.679,30.332,1.379
4,2022-01-07,15.462,12.495,3.798,17.000,23.450,3.391,3.286,24.440,29.291,...,5.372,4.942,28.659,2.376,15.912,64.204,1.554,21.626,30.243,1.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2022-12-23,15.292,12.397,4.423,16.799,24.572,3.268,3.260,24.245,27.546,...,5.223,4.942,NaN,2.183,16.912,65.810,1.222,22.825,30.390,1.340
248,2022-12-27,15.383,12.390,4.330,16.864,24.541,3.278,3.262,24.260,27.467,...,5.197,4.925,NaN,2.180,16.962,65.959,1.220,22.831,30.384,1.324
249,2022-12-28,15.485,12.371,4.323,16.889,24.589,3.268,3.261,24.250,27.539,...,5.157,4.903,NaN,2.184,16.934,65.641,1.218,22.796,30.337,1.326
250,2022-12-29,15.255,12.368,4.373,16.711,24.586,3.263,3.253,24.190,27.325,...,5.163,4.887,NaN,2.168,16.849,65.604,1.214,22.718,30.234,1.330


In [604]:
df_calendar

""
Date
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
...
2022-12-26
2022-12-27
2022-12-28


In [605]:
df_exrates['datum_index'] = df_exrates['Datum']
df_exrates

,Datum,AUD,BGN,BRL,CAD,CHF,CNY,DKK,EUR,GBP,...,RON,RUB,SEK,SGD,THB,TRY,USD,XDR,ZAR,datum_index
0,2022-01-03,15.818,12.690,3.906,17.210,23.931,3.441,3.337,24.820,29.502,...,5.016,29.362,2.410,16.189,65.897,1.646,21.860,30.595,1.382,2022-01-03
1,2022-01-04,15.784,12.655,3.857,17.211,23.903,3.441,3.328,24.750,29.599,...,5.002,29.146,2.407,16.166,65.862,1.635,21.944,30.744,1.367,2022-01-04
2,2022-01-05,15.764,12.567,3.831,17.076,23.722,3.414,3.305,24.580,29.426,...,4.970,28.697,2.397,16.040,65.439,1.613,21.739,30.359,1.372,2022-01-05
3,2022-01-06,15.547,12.542,3.810,16.977,23.600,3.398,3.297,24.530,29.345,...,4.962,28.356,2.375,15.944,64.656,1.578,21.679,30.332,1.379,2022-01-06
4,2022-01-07,15.462,12.495,3.798,17.000,23.450,3.391,3.286,24.440,29.291,...,4.942,28.659,2.376,15.912,64.204,1.554,21.626,30.243,1.384,2022-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2022-12-23,15.292,12.397,4.423,16.799,24.572,3.268,3.260,24.245,27.546,...,4.942,NaN,2.183,16.912,65.810,1.222,22.825,30.390,1.340,2022-12-23
248,2022-12-27,15.383,12.390,4.330,16.864,24.541,3.278,3.262,24.260,27.467,...,4.925,NaN,2.180,16.962,65.959,1.220,22.831,30.384,1.324,2022-12-27
249,2022-12-28,15.485,12.371,4.323,16.889,24.589,3.268,3.261,24.250,27.539,...,4.903,NaN,2.184,16.934,65.641,1.218,22.796,30.337,1.326,2022-12-28
250,2022-12-29,15.255,12.368,4.373,16.711,24.586,3.263,3.253,24.190,27.325,...,4.887,NaN,2.168,16.849,65.604,1.214,22.718,30.234,1.330,2022-12-29


In [606]:
df_all_dates = df_calendar.merge(df_exrates, how='left', left_index = True, right_on='datum_index')
# df_all_dates.set_index('datum_index', inplace=True)
df_all_dates.head(15)


,Datum,AUD,BGN,BRL,CAD,CHF,CNY,DKK,EUR,GBP,...,RON,RUB,SEK,SGD,THB,TRY,USD,XDR,ZAR,datum_index
0.0,2022-01-03,15.818,12.690,3.906,17.210,23.931,3.441,3.337,24.820,29.502,...,5.016,29.362,2.410,16.189,65.897,1.646,21.860,30.595,1.382,2022-01-03
1.0,2022-01-04,15.784,12.655,3.857,17.211,23.903,3.441,3.328,24.750,29.599,...,5.002,29.146,2.407,16.166,65.862,1.635,21.944,30.744,1.367,2022-01-04
2.0,2022-01-05,15.764,12.567,3.831,17.076,23.722,3.414,3.305,24.580,29.426,...,4.970,28.697,2.397,16.040,65.439,1.613,21.739,30.359,1.372,2022-01-05
3.0,2022-01-06,15.547,12.542,3.810,16.977,23.600,3.398,3.297,24.530,29.345,...,4.962,28.356,2.375,15.944,64.656,1.578,21.679,30.332,1.379,2022-01-06
4.0,2022-01-07,15.462,12.495,3.798,17.000,23.450,3.391,3.286,24.440,29.291,...,4.942,28.659,2.376,15.912,64.204,1.554,21.626,30.243,1.384,2022-01-07
NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-08
NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-09
5.0,2022-01-10,15.444,12.456,3.809,17.004,23.317,3.378,3.275,24.360,29.212,...,4.926,28.664,2.364,15.877,64.023,1.547,21.525,30.096,1.376,2022-01-10
6.0,2022-01-11,15.449,12.483,3.821,17.035,23.250,3.379,3.281,24.415,29.252,...,4.937,28.769,2.369,15.915,64.515,1.566,21.538,30.151,1.379,2022-01-11
7.0,2022-01-12,15.495,12.486,3.847,17.127,23.285,3.375,3.282,24.420,29.302,...,4.938,28.800,2.381,15.898,64.390,1.556,21.470,30.066,1.391,2022-01-12


In [607]:
# df_all_dates.ffill().head(15)
df_all_dates.ffill(inplace=True)
df_all_dates.set_index('datum_index', inplace=True)
df_all_dates.rename(columns={'Datum':'CNB_Date'}, inplace=True)
df_all_dates.head(15)

,CNB_Date,AUD,BGN,BRL,CAD,CHF,CNY,DKK,EUR,GBP,...,PLN,RON,RUB,SEK,SGD,THB,TRY,USD,XDR,ZAR
datum_index,,,,,,,,,,,,,,,,,,,,,
2022-01-03,2022-01-03,15.818,12.690,3.906,17.210,23.931,3.441,3.337,24.820,29.502,...,5.408,5.016,29.362,2.410,16.189,65.897,1.646,21.860,30.595,1.382
2022-01-04,2022-01-04,15.784,12.655,3.857,17.211,23.903,3.441,3.328,24.750,29.599,...,5.419,5.002,29.146,2.407,16.166,65.862,1.635,21.944,30.744,1.367
2022-01-05,2022-01-05,15.764,12.567,3.831,17.076,23.722,3.414,3.305,24.580,29.426,...,5.382,4.970,28.697,2.397,16.040,65.439,1.613,21.739,30.359,1.372
2022-01-06,2022-01-06,15.547,12.542,3.810,16.977,23.600,3.398,3.297,24.530,29.345,...,5.378,4.962,28.356,2.375,15.944,64.656,1.578,21.679,30.332,1.379
2022-01-07,2022-01-07,15.462,12.495,3.798,17.000,23.450,3.391,3.286,24.440,29.291,...,5.372,4.942,28.659,2.376,15.912,64.204,1.554,21.626,30.243,1.384
2022-01-08,2022-01-07,15.462,12.495,3.798,17.000,23.450,3.391,3.286,24.440,29.291,...,5.372,4.942,28.659,2.376,15.912,64.204,1.554,21.626,30.243,1.384
2022-01-09,2022-01-07,15.462,12.495,3.798,17.000,23.450,3.391,3.286,24.440,29.291,...,5.372,4.942,28.659,2.376,15.912,64.204,1.554,21.626,30.243,1.384
2022-01-10,2022-01-10,15.444,12.456,3.809,17.004,23.317,3.378,3.275,24.360,29.212,...,5.373,4.926,28.664,2.364,15.877,64.023,1.547,21.525,30.096,1.376
2022-01-11,2022-01-11,15.449,12.483,3.821,17.035,23.250,3.379,3.281,24.415,29.252,...,5.372,4.937,28.769,2.369,15.915,64.515,1.566,21.538,30.151,1.379


A ted to muzu zase dostat z tech sloupecku do radku

In [608]:
df_full_calendar_depivoted = df_all_dates.reset_index().melt(id_vars=['datum_index', 'CNB_Date'],var_name = 'currency', value_name='ex_rate'  )
df_full_calendar_depivoted

,datum_index,CNB_Date,currency,ex_rate
0,2022-01-03,2022-01-03,AUD,15.818
1,2022-01-04,2022-01-04,AUD,15.784
2,2022-01-05,2022-01-05,AUD,15.764
3,2022-01-06,2022-01-06,AUD,15.547
4,2022-01-07,2022-01-07,AUD,15.462
...,...,...,...,...
11941,2022-12-26,2022-12-23,ZAR,1.340
11942,2022-12-27,2022-12-27,ZAR,1.324
11943,2022-12-28,2022-12-28,ZAR,1.326
11944,2022-12-29,2022-12-29,ZAR,1.330


In [609]:
df_full_calendar_depivoted.to_csv('calendar_with_ex_rates_long.csv', index=False)

### Let's put the code into methods

In [610]:
import pandas as pd

def download_year(year:int, base_URL = 'https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/rok.txt', paramname = 'rok'):
    '''
    Download the year from CNB web as dataframe
    '''

    if pd.Timestamp.today().year < year:
        return None

    full_URL = f'{base_URL}?{paramname}={year}'
    df_exrates = pd.read_csv(full_URL, sep='|', header=0, parse_dates=[0], date_format = '%d.%m.%Y', decimal=',')

    # if df_exrates['Datum'].dtype == 'object':
    #     df_exrates['Datum'] = pd.to_datetime(df_exrates['Datum'], format = '%d.%m.%Y', errors = 'coerce')
    #     df_exrates = df_exrates[df_exrates['Datum'].notna()]

    if df_exrates['Datum'].dtype == 'object':
        ser_new_headers = pd.Series(df_exrates[df_exrates['Datum']== 'Datum'].index)
        print(f'WARNING: change in columns during the year {year} on lines {ser_new_headers}')
        skip_rows = 0
        partial_dfs = list()
        for hdr_line in ser_new_headers:
            df_exrates_partial = pd.read_csv(full_URL, sep='|', header=0, parse_dates=[0], date_format = '%d.%m.%Y', decimal=',', nrows=hdr_line-skip_rows, skiprows=skip_rows)
            partial_dfs.append(df_exrates_partial)
            skip_rows = hdr_line+1

        df_exrates_partial = pd.read_csv(full_URL, sep='|', header=0, parse_dates=[0], date_format = '%d.%m.%Y', decimal=',', skiprows=skip_rows)
        partial_dfs.append(df_exrates_partial)

        df_exrates =  pd.concat(partial_dfs, ignore_index=True)


    return df_exrates

def download_years(year_from,
                   year_to = 2100,
                   base_URL = 'https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/rok.txt',
                   paramname = 'rok'):
    '''
    Download years from/to a specific year from CNB web as dataframe
    '''

    year_to = min(pd.Timestamp.today().year, year_to)
    df_exrates_years = pd.DataFrame()
    exrates_years = list()
    for year in range(year_from, year_to + 1):
        df_year = download_year(year, base_URL, paramname)
        exrates_years.append(df_year)

    df_exrates_years = pd.concat(exrates_years, ignore_index=True)
    return df_exrates_years




In [611]:
df_exrates = download_years(2023)
df_exrates

,Datum,1 AUD,1 BGN,1 BRL,1 CAD,1 CHF,1 CNY,1 DKK,1 EUR,1 GBP,...,1 PLN,1 RON,1 SEK,1 SGD,100 THB,1 TRY,1 USD,1 XDR,1 ZAR,100 TRY
0,2023-01-02,15.400,12.385,4.237,16.668,24.496,3.281,3.250,24.175,27.272,...,5.164,4.894,2.165,16.878,65.491,1.209,22.630,30.118,1.330,NaN
1,2023-01-03,15.358,12.335,4.259,16.738,24.422,3.311,3.244,24.125,27.401,...,5.152,4.893,2.165,17.006,66.548,1.221,22.878,30.447,1.339,NaN
2,2023-01-04,15.550,12.284,4.161,16.729,24.409,3.293,3.230,24.025,27.276,...,5.147,4.877,2.152,16.907,66.658,1.210,22.672,30.148,1.347,NaN
3,2023-01-05,15.483,12.280,4.209,16.795,24.400,3.300,3.231,24.030,27.218,...,5.146,4.880,2.148,16.914,66.846,1.208,22.676,30.267,1.326,NaN
4,2023-01-06,15.430,12.299,4.294,16.786,24.387,3.340,3.235,24.055,27.189,...,5.124,4.879,2.137,16.993,67.322,1.221,22.911,30.549,1.320,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,2024-10-08,15.536,12.943,4.184,16.899,26.893,3.269,3.394,25.315,30.214,...,5.875,5.086,2.230,17.693,68.965,NaN,23.052,30.883,1.321,67.262
446,2024-10-09,15.564,12.961,4.176,16.919,26.975,3.274,3.399,25.350,30.273,...,5.898,5.095,2.231,17.731,69.125,NaN,23.135,31.036,1.311,67.569
447,2024-10-10,15.557,12.946,4.147,16.842,26.970,3.272,3.395,25.320,30.251,...,5.882,5.088,2.228,17.704,68.945,NaN,23.160,31.031,1.316,67.691
448,2024-10-11,15.578,12.933,4.152,16.788,26.979,3.273,3.390,25.295,30.217,...,5.896,5.084,2.227,17.708,69.453,NaN,23.123,30.947,1.323,67.498


In [612]:
def get_currencies(df_exrates:pd.DataFrame):
    df_currencies = pd.DataFrame()
    df_currencies['orig_name'] = df_exrates.columns[1:]
    df_currencies[['multiplier', 'currency']] = df_currencies['orig_name'].str.split(pat = ' ', expand=True)
    df_currencies.set_index('orig_name', inplace=True)
    return df_currencies


In [613]:
def rename_columns(df_exrates, df_currencies):
    new_column_names =  pd.Series(df_currencies['currency'],index=df_currencies.index).to_dict()
    # df_exrates.rename(columns=new_column_names, inplace=True)
    # df_exrates.set_index('Datum', inplace=True)
    return df_exrates.rename(columns=new_column_names).set_index('Datum')



In [614]:
def get_full_calendar(df_exrates:pd.DataFrame):
    min_date = df_exrates.index.min()
    max_date = df_exrates.index.max()
    df_calendar = pd.DataFrame({"Date": pd.date_range(min_date, max_date)})
    df_calendar.set_index('Date', inplace=True)

    if 'Datum' not in df_exrates.columns:
        df_exrates.reset_index(inplace=True)

    df_exrates['datum_index'] = df_exrates['Datum']
    df_all_dates = df_calendar.merge(df_exrates, how='left', left_index = True, right_on='datum_index')
    df_all_dates.ffill(inplace=True)
    df_all_dates.set_index('datum_index', inplace=True)
    df_all_dates.rename(columns={'Datum':'CNB_Date'}, inplace=True)
    return df_all_dates




In [615]:
def convert_columns_to_rows(df_full_calendar: pd.DataFrame)->pd.DataFrame:
    df_full_calendar_depivoted = df_full_calendar.reset_index().melt(id_vars=['datum_index', 'CNB_Date'],var_name = 'currency', value_name='ex_rate'  )
    return df_full_calendar_depivoted


In [616]:
def download_years_to_csv(year_from:int,
                          year_to:int = 2100,
                          fname:str = 'calendar_with_ex_rates.csv',
                          currency_fname:str = None,
                          long_format:bool = True,
                          base_URL:str = 'https://www.cnb.cz/cs/financni-trhy/devizovy-trh/kurzy-devizoveho-trhu/kurzy-devizoveho-trhu/rok.txt',
                          paramname:str = 'rok'):
    df_exrates = download_years(year_from=year_from,
                                year_to=year_to,
                                base_URL=base_URL,
                                paramname=paramname)
    df_currencies = get_currencies(df_exrates)
    if currency_fname:
        df_currencies.to_csv(currency_fname)

    df_exrates = rename_columns(df_exrates, df_currencies)
    df_full_calendar = get_full_calendar(df_exrates)
    if long_format:
        df_full_calendar_depivoted = convert_columns_to_rows(df_full_calendar)
        df_full_calendar_depivoted.to_csv(fname, index=False)
    else:
        df_full_calendar.to_csv(fname, index=True, index_label='datum_index')



In [617]:
if True:
    df_exrates = download_year(2009) # 2022 is strange, there is a row 42 in the file that contains header (again)
    # df_currencies = get_currencies(df_exrates)
    # df_exrates = rename_columns(df_exrates, df_currencies)
    # df_full_calendar = get_full_calendar(df_exrates)
# df_exrates.info()


df_exrates.head(50)


# min_date = df_exrates.index.min()

dtype: int64


,Datum,1 AUD,1 BGN,1 BRL,1 CAD,1 CHF,1 CNY,1 DKK,1 EEK,1 EUR,...,1 PLN,1 RON,100 RUB,1 SEK,1 SGD,100 THB,1 TRY,1 USD,1 XDR,1 ZAR
0,2009-01-02,13.493,13.715,8.275,15.833,18.037,2.836,3.601,1.715,26.830,...,6.444,6.675,65.789,2.473,13.242,55.682,12.552,19.347,29.799,2.051
1,2009-01-05,13.944,13.675,8.467,16.232,17.759,2.884,3.593,1.710,26.760,...,6.468,6.609,67.600,2.498,13.383,56.351,12.805,19.695,30.335,2.079
2,2009-01-06,14.097,13.496,8.939,16.653,17.566,2.898,3.543,1.688,26.405,...,6.569,6.496,67.947,2.498,13.394,56.491,13.039,19.809,30.098,2.121
3,2009-01-07,13.854,13.337,8.697,16.236,17.370,2.808,3.505,1.669,26.120,...,6.572,6.367,65.890,2.470,13.048,55.081,12.622,19.185,28.891,2.050
4,2009-01-08,13.506,13.390,8.468,16.012,17.529,2.813,3.514,1.674,26.190,...,6.495,6.296,65.367,2.445,12.971,55.038,12.336,19.222,29.318,1.994
5,2009-01-09,13.672,13.540,8.481,16.351,17.658,2.831,3.553,1.692,26.480,...,6.546,6.257,66.365,2.470,13.075,55.551,12.481,19.351,29.514,2.005
6,2009-01-12,13.640,13.591,8.684,16.586,17.711,2.903,3.566,1.699,26.580,...,6.567,6.216,63.908,2.470,13.359,56.964,12.603,19.846,30.469,1.981
7,2009-01-13,13.465,13.681,8.700,16.390,18.005,2.952,3.591,1.710,26.760,...,6.461,6.251,64.620,2.450,13.544,57.781,12.607,20.174,30.698,2.001
8,2009-01-14,13.622,13.765,8.809,16.679,18.238,2.991,3.614,1.721,26.930,...,6.499,6.278,64.363,2.460,13.694,58.551,12.772,20.440,30.968,2.035
9,2009-01-15,13.776,13.952,8.725,16.668,18.531,3.051,3.664,1.745,27.300,...,6.455,6.346,64.529,2.484,13.928,59.780,12.893,20.860,31.534,2.049


In [618]:
# download_years_to_csv(2021, fname= 'exrates_2021_2024.csv', long_format=True)
download_years_to_csv(2006, fname= 'exrates_test.csv', currency_fname = 'currencies.csv', long_format=True)

dtype: int64
dtype: int64
